<a href="https://www.kaggle.com/code/tarktunataalt/bike-sharing-demand-sarimax-hybrid-models?scriptVersionId=185957346" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# INTRODUCTION

This notebook focuses on developing SARIMAX and hybrid models to forecast bike-sharing demand. SARIMAX models are used to capture both seasonal and non-seasonal patterns in the time series data.

Before diving into these models, it is recommended to review the Exploratory Data Analysis (EDA) conducted in the previous notebook to gain a better understanding of the data:
* [ Exploratory Data Analysis (EDA)](https://www.kaggle.com/code/tarktunataalt/washington-dc-capital-bikeshare-eda)

Additionally, for further model improvements and comparisons, the following notebooks are available:
* [ STL and Hybrid Models](https://www.kaggle.com/code/tarktunataalt/bike-sharing-demand-stl-hybrid-models)
* [LSTM Models](https://www.kaggle.com/code/tarktunataalt/bike-sharing-demand-lstm-stl-models)

In [ ]:
libraries <- c(
  "psych",
  "dplyr",
  "magrittr",
  "ggplot2",
  "gridExtra",
  "grid",
  "patchwork",
  "lmtest",
  "zoo",
  "xgboost",
  "Metrics",
  "plotly",
  "knitr",
  "forecast",
  "randomForest",
  "gbm",
  "lightgbm",
  "keras",
  "caret",
  "dplyr",
  "nortsTest",
  "tseries",
  "urca",
  "reshape2",
  "catboost"
)

load_libraries <- function(libraries) {
  for (lib in libraries) {
    if (!require(lib, character.only = TRUE)) {
      suppressMessages(suppressWarnings(install.packages(lib, dependencies = TRUE)))
      suppressPackageStartupMessages(library(lib, character.only = TRUE))
    } else {
      suppressPackageStartupMessages(library(lib, character.only = TRUE))
    }
  }
}

load_libraries(libraries)

data <- read.csv("/kaggle/input/bike-sharing-dataset/day.csv", header = TRUE)
data %<>% select(-c("instant", "casual", "registered"))
data$date <- as.Date(data$dteday, format = "%Y-%m-%d")
data$dteday <- NULL


In [ ]:
data <- data[order(data$date), ]

train_size <- floor(0.8 * nrow(data))
train_indices <- 1:train_size
test_indices <- (train_size + 1):nrow(data)

train_data <- data[train_indices, ]
test_data <- data[test_indices, ]

print(dim(train_data))
print(dim(test_data))

In [ ]:
train_ts <- ts(train_data$cnt, start = c(2011, 1), frequency = 30)
test_ts <- ts(test_data$cnt, start = c(2011 + (train_size / 30), 1), frequency = 30)

# SARIMAX

In [ ]:
acf_plot <- ggAcf(train_ts, lag.max = 130) + ggtitle("ACF for Raw Data")
pacf_plot <- ggPacf(train_ts, lag.max = 130) + ggtitle("PACF for Raw Data")
acf_plot / pacf_plot

* **ACF (Autocorrelation Function)**:
The ACF plot shows significant autocorrelations at multiple lags, indicating that the raw data series is not stationary. The presence of spikes across all lags suggests a strong trend component in the data.

* **PACF (Partial Autocorrelation Function)**:
The PACF plot reveals significant partial autocorrelations at the initial lags, which then quickly drop off. This pattern implies that the series may have a trend and possibly seasonal components.

Given the consistent spikes at all lags in the ACF plot, it is clear that the raw data is non-stationary. To address this, we will apply a first-order non-seasonal differencing to remove the trend and achieve stationarity, which is essential for accurate time series modeling.

In [ ]:
diff1 <- diff(train_ts, 1)
acf_plot <- ggAcf(diff1, lag.max = 130) + ggtitle("ACF for First-Order Differenced Data")
pacf_plot <- ggPacf(diff1, lag.max = 130) + ggtitle("PACF for First-Order Differenced Dataa")
acf_plot / pacf_plot

* **ACF (Autocorrelation Function)**:
The ACF plot shows significant cut-offs after a few lags, indicating that the non-seasonal part of the series can be modeled as MA(2).

* **PACF (Partial Autocorrelation Function)**:
The PACF plot shows a slight decline after the initial lags.

* **Seasonal Component**:
Both the ACF and PACF plots show a significant spike at the 30th lag. This suggests that for the seasonal component, AR(1), MA(1), and ARMA(1,1) models should be considered.

Before finalizing these decisions, stationarity tests need to be conducted for both the non-seasonal and seasonal components. These tests will be performed in the subsequent sections.

In [ ]:
adf.test(diff1)

The Augmented Dickey-Fuller test was performed on the first-order differenced data. The test statistic is -12.859 with a p-value of 0.01. Since the p-value is less than 0.05, we reject the null hypothesis that the data has a unit root, indicating that the data is stationary. Consequently, the result supports that the data is stationary.

In [ ]:
summary(ur.kpss(diff1, type = "tau"))


The KPSS Unit Root Test was performed on the first-order differenced data. The test statistic is 0.0164.

Since the test statistic (0.0164) is less than the critical value at the 1% significance level (0.216), we fail to reject the null hypothesis that the data is stationary. Consequently, the result supports that the data is stationary.

In [ ]:
seasonal.test(diff1, seasonal = c("ocsb","ch","hegy"), alpha = 0.05)


The OCSB test was performed on the seasonal component of the data. The test statistic is -17.2611, with a 5% critical value of -1.7436.

Since the test statistic (-17.2611) is less than the 5% critical value (-1.7436), we reject the null hypothesis that the seasonal component has a unit root, indicating that the seasonal component is stationary. Consequently, the result supports that the seasonal component of the data is stationary.

In [ ]:
exog <- train_data[, !names(train_data) %in% c("cnt", "date")]
exog$season <- as.numeric(exog$season)
exog$yr <- as.numeric(exog$yr)
exog$mnth <- as.numeric(exog$mnth)
exog$holiday <- as.numeric(exog$holiday)
exog$weekday <- as.numeric(exog$weekday)
exog$workingday <- as.numeric(exog$workingday)
exog$weathersit <- as.numeric(exog$weathersit)
exog_matrix <- as.matrix(exog)

In [ ]:
sarimax_model1 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(1, 0, 0), period = 30), xreg = exog_matrix)
sarimax_model2 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(0, 0, 1), period = 30), xreg = exog_matrix)
sarimax_model3 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(1, 0, 1), period = 30), xreg = exog_matrix)

In [ ]:
coeftest(sarimax_model1)
coeftest(sarimax_model2)
coeftest(sarimax_model3)


The coefficients for these models were tested for statistical significance. The results showed that the seasonal components in all three models were not statistically significant. This implies that the seasonal part of the models did not contribute significantly to the model's performance.

Given that the seasonal components are not significant, we will introduce a seasonal differencing term to account for potential seasonality in the data. Additionally, all non-significant parameters will be dropped to refine the model. This process will help in simplifying the model and improving its interpretability while maintaining its predictive performance.

In [ ]:
exog %<>% select(c("season","yr","weekday", "weathersit", "hum", "windspeed"))
exog_matrix <- as.matrix(exog)

In [ ]:
diff130 <- diff(diff1, 30)
acf_plot <- ggAcf(diff1, lag.max = 130) + ggtitle("ACF for First-Order Differenced and First-Order Seasonal Differenced Data")
pacf_plot <- ggPacf(diff1, lag.max = 130) + ggtitle("PACF for First-Order Differenced and First-Order Seasonal Differenced Data")
acf_plot / pacf_plot

* **ACF (Autocorrelation Function)**:
The ACF plot for the first-order differenced and first-order seasonal differenced data shows significant cut-offs after a few lags, indicating that the non-seasonal part of the series can be modeled as MA(2).

* **PACF (Partial Autocorrelation Function)**:
The PACF plot for the same data reveals a slight decline after the initial lags.


For the seasonal differenced data, the same observation holds: both the ACF and PACF plots show a significant spike at the 30th lag. This suggests that for the seasonal component, AR(1), MA(1), and ARMA(1,1) models should be considered.

Thus, we can apply similar modeling strategies for both the non-seasonal and seasonal components, considering AR(1), MA(1), and ARMA(1,1) models for the seasonal part of the differenced data.

In [ ]:
sarimax_model1 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(1, 1, 0), period = 30), xreg = exog_matrix)
sarimax_model2 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(0, 1, 1), period = 30), xreg = exog_matrix)
sarimax_model3 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(1, 1, 1), period = 30), xreg = exog_matrix)

In [ ]:
coeftest(sarimax_model1)
coeftest(sarimax_model2)
coeftest(sarimax_model3)

The coefficients for these models were tested for statistical significance. The results indicated the following:

In Model 3 (SARIMA(0,1,2)(1,1,1)[30]), the sar1 parameter was not statistically significant (p-value = 0.404), indicating that the seasonal autoregressive term does not contribute significantly to the model. Therefore, this parameter will not be considered further.
In all models, the year variable (yr) was not statistically significant (p-values > 0.05), suggesting that this variable does not have a significant impact on the model and will be dropped.
Based on these findings, we will refine and re-evaluate the first two models by excluding the year variable and any other non-significant parameters:

Refined Model 1: SARIMA(0,1,2)(1,1,0)[30] without year
Refined Model 2: SARIMA(0,1,2)(0,1,1)[30] without year
These refined models will be further analyzed to ensure improved performance and interpretability.

In [ ]:
exog %<>% select(c("season","weekday", "weathersit", "hum", "windspeed"))
exog_matrix <- as.matrix(exog)

In [ ]:
automodel <- auto.arima(train_ts, stepwise = TRUE, trace = TRUE, seasonal = TRUE, approximation = FALSE, allowdrift = TRUE, allowmean = TRUE, xreg = exog_matrix)
sarimax_model1 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(1, 1, 0), period = 30), xreg = exog_matrix)
sarimax_model2 <- Arima(train_ts, order = c(0, 1, 2), seasonal = list(order = c(0, 1, 1), period = 30), xreg = exog_matrix)

To avoid performance loss, all variables in the exogenous matrix were made significant before selecting the best model using the auto.arima function. The best model suggested by auto.arima was Regression with ARIMA(1,1,1) errors.

However, while auto.arima provided a non-seasonal ARIMA model recommendation, seasonality can still be captured through the seasonal variable in the exogenous matrix. This means that although auto.arima did not explicitly identify a seasonal component, the presence of the seasonal variable allows the model to account for any underlying seasonal patterns in the data.

Other identified SARIMAX models will also be evaluated to ensure comprehensive model performance assessment.

In [ ]:
coeftest(automodel)
coeftest(sarimax_model1)
coeftest(sarimax_model2)

* **Auto.ARIMA Model: Regression with ARIMA(1,1,1)**
The auto.arima function suggested a non-seasonal ARIMA model. In this model, the ar1 and ma1 terms are highly significant, indicating a strong autoregressive and moving average component. The drift term is marginally significant, while the season term is near the significance threshold (p-value = 0.080), suggesting that the model might be able to capture seasonality to some extent through the exogenous season variable.

* **SARIMA(0,1,2)(1,1,0)[30]**
The ma1, ma2, and sar1 terms are highly significant, indicating a strong moving average and seasonal autoregressive component. The season variable is significant at the 5% level, suggesting that this model captures the seasonality better than the auto.arima model. Other significant variables include weekday, weathersit, hum, and windspeed.

* **SARIMA(0,1,2)(0,1,1)[30]**

The ma1, ma2, and sma1 terms are highly significant, indicating a strong moving average and seasonal moving average component. The season variable is significant at the 5% level, similar to the previous SARIMA model. This suggests that this model also captures seasonality effectively. Other significant variables include weekday, weathersit, hum, and windspeed.

* **Summary**
The auto.arima function did not suggest a seasonal model, and while the season variable in this model was near the significance threshold, it was not definitively significant. In contrast, the SARIMA models with seasonal components showed that the season variable was significant, indicating that these models can capture seasonality more effectively. Based on these results, it is clear that incorporating seasonal differencing and seasonal components into the SARIMA models provides a better fit for the data. Therefore, the refined SARIMA models will be further evaluated and compared to ensure optimal model performance.

In [ ]:
checkresiduals(sarimax_model1)

* Residuals are randomly distributed around zero, indicating a good fit overall, though some large residuals suggest potential outliers.
* Most autocorrelations are within significance bounds, indicating that residuals are mostly white noise, with a few exceptions.
* Residuals are approximately normally distributed, supporting the model's assumptions, with a symmetric spread around zero.

Overall, the model fits well, capturing the main data patterns and leaving mostly white noise residuals.

In [ ]:
tsdiag(sarimax_model1)

* The standardized residuals are mostly within the range of -2 to 2, indicating that the residuals are generally well-behaved and there are no obvious patterns or large outliers.
* The ACF plot of the residuals shows that most autocorrelations are within the significance bounds, suggesting that the residuals behave like white noise. This indicates that the model has effectively captured the structure in the data.
* The p-values for the Ljung-Box test at various lags are not sufficiently high, indicating that there might still be some autocorrelation remaining in the residuals. This suggests that while the model fits reasonably well, there may be room for improvement. Consequently, exploring alternative models may be necessary to better address these remaining autocorrelations.

Overall, the residual analysis suggests that the model fits the data well, but the p-values indicate that there might still be some remaining autocorrelation that could be addressed. Therefore, considering alternative models may help in improving the model's performance further.

In [ ]:
Box.test(sarimax_model1$residuals,lag=7)
Box.test(sarimax_model1$residuals,lag=15)
Box.test(sarimax_model1$residuals,lag=30)
Box.test(sarimax_model1$residuals,lag=60)
Box.test(sarimax_model1$residuals,lag=90)

The p-values significantly decrease at lags that are multiples of 30, approaching 0, indicating the presence of strong seasonal autocorrelation at these lags. This suggests that the current SARIMAX model may not fully capture the seasonal patterns in the data. Consequently, there is a need to explore alternative models or further refine the current model to better address these seasonal autocorrelations.

In [ ]:
shapiro.test(sarimax_model1$residuals)

The Shapiro-Wilk test results indicate that the residuals do not follow a normal distribution. This suggests that the model may not be adequately capturing all the underlying patterns in the data. The lack of normality in the residuals points to potential issues in model specification. Consequently, it may be necessary to explore alternative models to better fit the data and address these normality issues.

In [ ]:
checkresiduals(sarimax_model2)

* The residuals are randomly distributed around zero, indicating a generally good fit. However, some larger residuals suggest periods of higher variability or potential outliers.
* The ACF plot shows that most autocorrelations are within the significance bounds, indicating that the residuals are mostly white noise. This suggests that the model has effectively captured the main patterns in the data.
* The histogram of residuals, with an overlaid normal distribution curve, indicates that the residuals are approximately normally distributed. However, some deviations from normality are present, particularly in the tails.

Overall, the residual analysis suggests that the Regression with ARIMA(0,1,2)(0,1,1)[30] errors model fits the data well, capturing the main patterns and leaving mostly white noise residuals. However, the presence of larger residuals and deviations from normality indicate that exploring alternative models may help to better address these issues.

In [ ]:
tsdiag(sarimax_model2)

* The standardized residuals are mostly within the range of -2 to 2, indicating that the residuals are generally well-behaved with no obvious patterns or large outliers.
* The ACF plot of the residuals shows that most autocorrelations are within the significance bounds, suggesting that the residuals behave like white noise. This indicates that the model has effectively captured the structure in the data.
* The p-values for the Ljung-Box test at various lags occasionally decrease but remain well above the threshold value. This indicates that, despite some fluctuations, there is no significant autocorrelation remaining in the residuals.

Overall, the residual analysis suggests that the model fits the data well, capturing the main patterns and leaving mostly white noise residuals. Although the p-values for the Ljung-Box test fluctuate, they consistently remain above the significance threshold, indicating that significant autocorrelation is not present.

In [ ]:
Box.test(sarimax_model2$residuals,lag=7)
Box.test(sarimax_model2$residuals,lag=15)
Box.test(sarimax_model2$residuals,lag=30)
Box.test(sarimax_model2$residuals,lag=60)
Box.test(sarimax_model2$residuals,lag=90)

The p-values for the Box-Pierce test at all lags remain above the threshold value, indicating that there is no significant autocorrelation remaining in the residuals at these lags. This suggests that the SARIMAX(0,1,2)(0,1,1)[30] model effectively captures the autocorrelation structure of the data. Although the p-values fluctuate, they consistently stay above the significance threshold, supporting the adequacy of the model in handling autocorrelation.

In [ ]:
shapiro.test(sarimax_model2$residuals)

The Shapiro-Wilk test results indicate that the residuals are far from the threshold for normal distribution, suggesting that the residuals do not follow a normal distribution. Despite this, the SARIMAX(0,1,2)(0,1,1)[30] model provides the best overall validity compared to other models evaluated. It effectively captures the autocorrelation structure of the data, as indicated by the Box-Pierce test results, and manages to handle the data patterns adequately despite the residuals not being perfectly normally distributed. Thus, this model is considered the most valid among the tested models.

In [ ]:
exog_matrix_train <- data.matrix(train_data[, c("season", "weekday", "weathersit", "hum", "windspeed")])
exog_matrix_test <- data.matrix(test_data[, c("season", "weekday", "weathersit", "hum", "windspeed")])

sarimax_forecast <- forecast(sarimax_model2, xreg = exog_matrix_test, h = nrow(test_data))

sarimax_pred <- sarimax_forecast$mean
actual_values <- test_data$cnt

residuals_train <- residuals(sarimax_model2)
residuals_test <- actual_values - sarimax_pred


In [ ]:
train_df <- data.frame(date = train_data$date, cnt = train_data$cnt, type = "Train")
test_df <- data.frame(date = test_data$date, cnt = test_data$cnt, type = "Test")
sarimax_forecast_df <- data.frame(date = test_data$date, cnt = sarimax_pred, type = "SARIMAX Forecast")

ggplot() +
  geom_line(data = train_df, aes(x = date, y = cnt, color = 'Train'), size = 0.5) +
  geom_line(data = test_df, aes(x = date, y = cnt, color = 'Actual Test'), size = 0.5) +
  geom_line(data = sarimax_forecast_df, aes(x = date, y = cnt, color = 'SARIMAX (0,1,2) (1,0,0)[30] Forecast'), size = 0.5,) +
  labs(title = 'SARIMAX (0,1,2) (1,0,0)[30] vs Actual',
       x = 'Date',
       y = 'Count') +
scale_color_manual(values = c('Train' = 'blue', 'Actual Test' = 'red', 'SARIMAX (0,1,2) (1,0,0)[30] Forecast' = 'green'),
                     breaks = c('Train', 'Actual Test', 'SARIMAX (0,1,2) (1,0,0)[30] Forecast')) +  theme_minimal() +
  theme(legend.position = "bottom")+
  guides(color = guide_legend(title = NULL))

The green line representing the SARIMAX model's forecast initially follows the actual test data (red line) quite closely. This indicates that the SARIMAX model captures the fundamental patterns in the data well at the start. However, the model struggles to predict the subsequent downward trend, leading to deviations in its forecasts.

Overall, the plot shows that the SARIMAX(0,1,2)(1,0,0)[30] model makes reasonable predictions initially but fails to anticipate the reverse trend accurately. This suggests that while the model captures some of the seasonal and trend components in the data, it may require further refinement to improve its predictive performance for unexpected changes.

The aim is to enhance the forecasting performance of the SARIMAX model by testing various hybrid models. These hybrid models will combine the residuals from the SARIMAX model with different machine learning techniques, including XGBoost, CatBoost, LightGBM, GBM, and RandomForest. The goal is to apply these advanced regression algorithms to the residuals left by the SARIMAX model, thereby capturing any remaining patterns or nonlinearities that the initial model might have missed. Each machine learning algorithm brings its unique strengths: XGBoost's robustness and scalability, CatBoost's ability to handle categorical features, LightGBM's speed and efficiency, GBM's simplicity, and RandomForest's capacity to deal with overfitting. By combining the time-series capabilities of SARIMAX with the powerful predictive performance of these machine learning models, the aim is to achieve more accurate and reliable forecasts.

# SARIMAX AND HYBRID MODELS

In [ ]:
sarimax_rmse <- rmse(actual_values, sarimax_pred)

# XGBoost
dtrain <- xgb.DMatrix(data = exog_matrix_train, label = residuals_train)
dtest <- xgb.DMatrix(data = exog_matrix_test)



xgboost_model <- xgboost(data = dtrain, nrounds = 100, objective = "reg:squarederror", verbose = 0)
xgboost_residuals <- predict(xgboost_model, dtest)
xgboost_pred <- sarimax_pred + xgboost_residuals
xgboost_rmse <- rmse(actual_values, xgboost_pred)

# CatBoost
catboost_pool_train <- catboost.load_pool(data = exog_matrix_train, label = residuals_train)
catboost_pool_test <- catboost.load_pool(data = exog_matrix_test)

catboost_model <- catboost.train(catboost_pool_train, params = list(iterations = 100, loss_function = 'RMSE', verbose = 0))
catboost_residuals <- catboost.predict(catboost_model, catboost_pool_test)
catboost_pred <- sarimax_pred + catboost_residuals
catboost_rmse <- rmse(actual_values, catboost_pred)

# LightGBM
lightgbm_train <- lgb.Dataset(data = exog_matrix_train, label = residuals_train)
lightgbm_params <- list(objective = "regression", metric = "rmse")

lightgbm_model <- lgb.train(params = lightgbm_params, data = lightgbm_train, nrounds = 100, verbose = 0)
lightgbm_residuals <- predict(lightgbm_model, exog_matrix_test)
lightgbm_pred <- sarimax_pred + lightgbm_residuals
lightgbm_rmse <- rmse(actual_values, lightgbm_pred)

# GBM
gbm_model <- gbm(residuals_train ~ ., data = as.data.frame(exog_matrix_train), distribution = "gaussian", n.trees = 100, interaction.depth = 3, shrinkage = 0.1, verbose = FALSE)
gbm_residuals <- predict(gbm_model, as.data.frame(exog_matrix_test), n.trees = 100)
gbm_pred <- sarimax_pred + gbm_residuals
gbm_rmse <- rmse(actual_values, gbm_pred)

# RandomForest
rf_model <- randomForest(residuals_train ~ ., data = as.data.frame(exog_matrix_train), ntree = 100)
rf_residuals <- predict(rf_model, as.data.frame(exog_matrix_test))
rf_pred <- sarimax_pred + rf_residuals
rf_rmse <- rmse(actual_values, rf_pred)


In [ ]:
rmse_values <- data.frame(
  Model = c("SARIMAX", "SARIMAX + XGBoost", "SARIMAX + CatBoost", "SARIMAX + LightGBM", "SARIMAX + GBM", "SARIMAX + RandomForest"),
  RMSE = c(sarimax_rmse, xgboost_rmse, catboost_rmse, lightgbm_rmse, gbm_rmse, rf_rmse)
)

kable(rmse_values, format = "markdown", col.names = c("Model", "RMSE"))


The table presents the RMSE values for the baseline SARIMAX model and its hybrid versions with various machine learning techniques. The SARIMAX + XGBoost hybrid model achieves the lowest RMSE (2539.241), indicating that XGBoost effectively captures additional patterns in the residuals, thereby improving forecast accuracy. Other hybrid models, such as SARIMAX + CatBoost (2588.540), SARIMAX + LightGBM (2577.548), SARIMAX + GBM (2576.968), and SARIMAX + RandomForest (2565.998), also show improvements over the baseline SARIMAX model (2599.479). These results suggest that integrating SARIMAX with advanced machine learning techniques generally enhances the model's performance by better addressing residual patterns, with XGBoost and RandomForest standing out as particularly effective. Overall, the hybrid approach demonstrates significant potential in achieving more accurate and reliable forecasts.

In [ ]:
train_df <- data.frame(date = train_data$date, cnt = train_data$cnt, type = "Train")
test_df <- data.frame(date = test_data$date, cnt = test_data$cnt, type = "Test")
xgboost_forecast_df <- data.frame(date = test_data$date, cnt = xgboost_pred, type = "SARIMAX + XGBoost Forecast")

ggplot() +
  geom_line(data = train_df, aes(x = date, y = cnt, color = 'Train'), size = 0.5) +
  geom_line(data = test_df, aes(x = date, y = cnt, color = 'Actual Test'), size = 0.5) +
  geom_line(data = xgboost_forecast_df, aes(x = date, y = cnt, color = 'SARIMAX + XGBoost Forecast'), size = 0.5) +
  labs(title = 'SARIMAX Model and XGBoost Hybrid Model Forecast vs Actual',
       x = 'Date',
       y = 'Count') +
    scale_color_manual(values = c('Train' = 'blue', 'Actual Test' = 'red', 'SARIMAX + XGBoost Forecast' = 'green'),
                     breaks = c('Train', 'Actual Test', 'SARIMAX + XGBoost Forecast')) +  theme_minimal() +
  theme(legend.position = "bottom")+
  guides(color = guide_legend(title = NULL))



The green line representing the SARIMAX + XGBoost hybrid model's forecast closely follows the actual test data (red line), indicating that the hybrid model captures the fundamental patterns in the data well. Initially, the forecast aligns closely with the actual values, demonstrating the model's ability to track the data accurately. However, similar to the pure SARIMAX model, the hybrid model struggles to predict the subsequent downward trend accurately, leading to some deviations.

Overall, the plot shows that the SARIMAX + XGBoost hybrid model provides a reasonable forecast, closely aligning with the actual test data and improving the prediction accuracy compared to the SARIMAX model alone. This indicates that integrating XGBoost with SARIMAX enhances the model's ability to capture additional patterns in the data, resulting in more accurate and reliable forecasts. Nonetheless, the model's difficulty in anticipating the reverse trend suggests that there is still room for further refinement.